# AI-Project 4
Baosheng Feng, Jiahang Chen

Project-A2

In [1]:
import numpy as np
import copy

In [2]:
#import the grid
# "-" is nan in the grid
path = "gridA.2.csv"
grid = np.genfromtxt(path, delimiter=',')
print(grid)
print(grid.shape)

[[ 1000.     0.     0.     0.     0.     0.]
 [    0.     0.    nan     0.     0.     0.]
 [ -800.     0.     0.     0.     0.     0.]
 [    0.     0.     0.     0.     0.     0.]
 [   nan     0.     0.     0.     0.     0.]
 [    0.     0.     0.    nan     0.     0.]
 [    0.     0.     0.     0.     0.     0.]
 [    0.     0.     0.     0.     0.     0.]
 [    0.    nan     0.     0.     0.    nan]
 [    0.     0.     0.     0.     0.     0.]
 [    0.     0.     0.     0.     0.     0.]
 [    0.     0.     0.     0.    nan     0.]
 [    0.     0.     0.     0.     0.  -800.]
 [   nan     0.     0.     0.     0.     0.]
 [    0.     0.    nan     0.     0.     0.]
 [    0.     0.     0.     0.     0.  1000.]]
(16, 6)


In [3]:
#parameters
row_dim = 16
col_dim = 6
discount = 0.9
reward = -0.01
action_space = [["U",0.60],["S",0.15],["L",0.1],["R",0.15]]

In [4]:
#used to get the sum of the whole grid
#used the sum to find if the grid changed
def grid_sum(grid):
    sum = 0
    for row in grid:
        for element in row:
            if np.isnan(element) == False :
                sum +=element
    
    return sum

In [5]:
def tran_value(value):
    return reward+discount*value

In [6]:
delta = 0.00001
delta_grid = 1
k = 1

In [7]:
#get the policy result
#A: 1000
#B: -800
#-: the wall
policy = np.zeros((16,6),dtype = str)

for i in range(16):
    for j in range(6):
        if grid[i][j] == 1000:
            policy[i][j] = "A"
        if grid[i][j] == -800:
            policy[i][j] = "B"
        if np.isnan(grid[i][j]):
            policy[i][j] = "-"

print(policy)

[['A' '' '' '' '' '']
 ['' '' '-' '' '' '']
 ['B' '' '' '' '' '']
 ['' '' '' '' '' '']
 ['-' '' '' '' '' '']
 ['' '' '' '-' '' '']
 ['' '' '' '' '' '']
 ['' '' '' '' '' '']
 ['' '-' '' '' '' '-']
 ['' '' '' '' '' '']
 ['' '' '' '' '' '']
 ['' '' '' '' '-' '']
 ['' '' '' '' '' 'B']
 ['-' '' '' '' '' '']
 ['' '' '-' '' '' '']
 ['' '' '' '' '' 'A']]


In [8]:
#the result
while delta_grid > delta:
    
    grid_old_sum = grid_sum(grid)
    
    for i in range(row_dim):
        for j in range(col_dim):
                        
            # start from lower left
            row = 15 - i
            col = j
            
            current = grid[row][col]
            
            #skip wall, 1000, -800
            if current == 1000 or current == -800 or np.isnan(current):
                continue
            
            #north value
            if row == 0 or np.isnan(grid[row-1][col]):
                n_value = current
            else:
                n_value = grid[row-1][col]
            
            #south value
            if row == 15 or np.isnan(grid[row+1][col]):
                s_value = current
            else:
                s_value = grid[row+1][col]
            
            #west value
            if col == 0 or np.isnan(grid[row][col-1]):
                w_value = current
            else:
                w_value = grid[row][col-1]
            
            #east value
            if col == 5 or np.isnan(grid[row][col+1]):
                e_value = current
            else:
                e_value = grid[row][col+1]
                
            
            
            move_north = 0.60*tran_value(n_value) + 0.15*tran_value(e_value) + 0.1*tran_value(w_value) + 0.15*tran_value(s_value)
            move_south = 0.60*tran_value(s_value) + 0.15*tran_value(w_value) + 0.1*tran_value(e_value) + 0.15*tran_value(n_value)
            move_west = 0.60*tran_value(w_value) + 0.15*tran_value(n_value) + 0.1*tran_value(s_value) + 0.15*tran_value(e_value)
            move_east = 0.60*tran_value(e_value) + 0.15*tran_value(s_value) + 0.1*tran_value(n_value) + 0.15*tran_value(w_value)
            
            grid[row][col] = max(move_north, move_south, move_west, move_east)
            
            if move_north == grid[row][col]:
                policy[row][col] = "N"
            if move_south == grid[row][col]:
                policy[row][col] = "S"
            if move_west == grid[row][col]:
                policy[row][col] = "W"
            if move_east == grid[row][col]:
                policy[row][col] = "E"
            
                        
    grid_new_sum = grid_sum(grid)
    
    print("K: " + str(k))
    k +=1
    np.set_printoptions(precision=2)
    print(grid)
    print(policy)
    
    delta_grid = grid_new_sum - grid_old_sum
            
        

K: 1
[[  1.00e+03   5.60e+02   3.03e+02   1.63e+02   8.82e+01   4.76e+01]
 [  4.32e+02   2.27e+02        nan  -6.56e-02  -2.51e-02  -1.47e-02]
 [ -8.00e+02  -7.29e+01  -6.65e+00  -6.17e-01  -6.79e-02  -1.79e-02]
 [ -7.20e+01  -6.49e+00  -5.96e-01  -6.51e-02  -1.76e-02  -1.33e-02]
 [       nan  -1.11e-02  -1.27e-02  -1.11e-02  -1.25e-02  -1.28e-02]
 [ -1.10e-02  -1.27e-02  -1.29e-02        nan  -1.12e-02  -1.27e-02]
 [ -1.10e-02  -1.25e-02  -1.28e-02  -1.29e-02  -1.29e-02  -1.27e-02]
 [ -1.10e-02  -1.10e-02  -1.25e-02  -1.28e-02  -1.29e-02  -1.12e-02]
 [ -1.10e-02        nan  -1.12e-02  -1.27e-02  -1.29e-02        nan]
 [ -1.10e-02  -1.27e-02  -1.29e-02  -1.29e-02  -1.27e-02  -6.02e-01]
 [ -1.10e-02  -1.27e-02  -1.29e-02  -1.29e-02  -1.12e-02  -6.50e+00]
 [ -1.09e-02  -1.27e-02  -1.28e-02  -1.29e-02        nan  -7.20e+01]
 [ -1.00e-02  -1.24e-02  -1.26e-02  -1.28e-02   1.30e+01  -8.00e+02]
 [       nan  -1.11e-02  -1.10e-02  -1.25e-02   1.57e+02   2.26e+02]
 [ -1.09e-02  -1.25e-02      

 [  323.55   396.99   504.24   625.17   786.98  1000.  ]]
[['A' 'W' 'W' 'W' 'W' 'W']
 ['N' 'N' '-' 'N' 'W' 'W']
 ['B' 'N' 'W' 'N' 'N' 'N']
 ['E' 'N' 'N' 'N' 'N' 'N']
 ['-' 'N' 'N' 'N' 'N' 'N']
 ['E' 'N' 'N' '-' 'N' 'N']
 ['N' 'N' 'N' 'W' 'N' 'N']
 ['S' 'N' 'S' 'S' 'S' 'W']
 ['S' '-' 'S' 'S' 'S' '-']
 ['S' 'S' 'S' 'S' 'S' 'W']
 ['S' 'S' 'S' 'S' 'W' 'W']
 ['E' 'S' 'S' 'S' '-' 'N']
 ['E' 'S' 'S' 'S' 'S' 'B']
 ['-' 'E' 'E' 'S' 'S' 'S']
 ['S' 'S' '-' 'E' 'S' 'S']
 ['E' 'E' 'E' 'E' 'E' 'A']]
K: 15
[[ 1000.     785.46   633.87   497.54   394.95   326.01]
 [  563.51   598.69      nan   387.31   322.95   276.01]
 [ -800.     318.88   268.6    299.59   259.69   227.79]
 [   72.77   233.27   221.41   234.51   207.96   186.28]
 [     nan   185.23   179.93   189.01   165.42   150.39]
 [  111.54   142.64   142.45      nan   127.6    119.08]
 [   89.5    109.44   109.71    92.31    98.34    92.75]
 [   73.56    86.84    93.93    94.51    86.15    74.5 ]
 [   81.59      nan   110.35   115.41   100.61 

## References:
* http://aima.cs.berkeley.edu/python/mdp.html
* https://www.datahubbs.com/reinforcement-learning-markov-decision-processes/